## create database

In [1]:
from sqlalchemy import Column, Integer, String, Text, Numeric, DateTime, Index
from sqlalchemy.orm import declarative_base
from datetime import datetime

Base = declarative_base()

class Document(Base):
    __tablename__ = 'documents'

    id = Column(Integer, primary_key=True, index=True)
    vendor = Column(String(100), index=True)
    data = Column(Text)
    amount = Column(Numeric(12, 2), index=True)
    category = Column(String(50), index=True)
    created_at = Column(DateTime, default=datetime.utcnow)

    __table_args__ = (
        Index('idx_vendor_category', 'vendor', 'category'),
    )


## connect database

In [2]:
from sqlalchemy import create_engine


# Local SQLite database file
DATABASE_URL = "sqlite:///./local_documents.db"

engine = create_engine(DATABASE_URL, echo=True)
Base.metadata.create_all(bind=engine)

print("✅ Local SQLite database and tables created successfully.")


2025-07-21 20:42:04,484 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-21 20:42:04,486 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("documents")
2025-07-21 20:42:04,487 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-21 20:42:04,489 INFO sqlalchemy.engine.Engine COMMIT
✅ Local SQLite database and tables created successfully.


## insert database

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# from models import Document
from datetime import datetime

DATABASE_URL = "sqlite:///./local_documents.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)

session = SessionLocal()

# Insert a sample document
new_doc = Document(
    vendor="Apple",
    data="Invoice data for MacBook",
    amount=159999.99,
    category="Computers",
    created_at=datetime.utcnow()
)

session.add(new_doc)
session.commit()
session.refresh(new_doc)

print(f"✅ Inserted Document ID: {new_doc.id}")
session.close()


✅ Inserted Document ID: 1003


In [40]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime, timedelta
import random
import string

# Assuming your Document model is already defined
# from models import Document

# Setup DB
DATABASE_URL = "sqlite:///./local_documents.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()

# Sample data for randomization
vendors = ["Apple", "Microsoft", "Amazon", "Google", "Samsung", "Sony", "Lenovo", "Dell"]
categories = ["Computers", "Utilities", "Electricity", "Internet", "Software", "Accessories"]

def random_string(length=20):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def random_date():
    # Generate a random date within the past 2 years
    days_ago = random.randint(0, 730)
    return datetime.utcnow() - timedelta(days=days_ago)

documents = []

for _ in range(1000):
    doc = Document(
        vendor=random.choice(vendors),
        data=f"Invoice data for {random_string(10)}",
        amount=round(random.uniform(50, 10000), 2),
        category=random.choice(categories),
        created_at=random_date()
    )
    documents.append(doc)

# Bulk insert
session.bulk_save_objects(documents)
session.commit()
print("✅ Inserted 1000 random sample documents")
session.close()


✅ Inserted 1000 random sample documents


In [4]:
def get_all_documents():
    session = SessionLocal()
    try:
        documents = session.query(Document).all()
        result = []
        for doc in documents:
            result.append({
                "id": doc.id,
                "vendor": doc.vendor,
                "data": doc.data,
                "amount": float(doc.amount),
                "category": doc.category,
                "created_at": doc.created_at.isoformat()
            })
        return result
    except Exception as e:
        print("❌ Error retrieving documents:", e)
        return []
    finally:
        session.close()

In [5]:
get_all_documents()

[{'id': 1,
  'vendor': 'Google',
  'data': 'Invoice data for 4azX5DZa2B',
  'amount': 3849.86,
  'category': 'Computers',
  'created_at': '2024-06-26T09:55:33.619896'},
 {'id': 2,
  'vendor': 'Sony',
  'data': 'Invoice data for 3S39BktxxI',
  'amount': 2279.34,
  'category': 'Accessories',
  'created_at': '2024-03-03T09:55:33.619896'},
 {'id': 3,
  'vendor': 'Sony',
  'data': 'Invoice data for SjhhWtDMLc',
  'amount': 8448.37,
  'category': 'Accessories',
  'created_at': '2023-08-07T09:55:33.619896'},
 {'id': 4,
  'vendor': 'Apple',
  'data': 'Invoice data for Om9k5mT33T',
  'amount': 3443.61,
  'category': 'Software',
  'created_at': '2023-12-09T09:55:33.619896'},
 {'id': 5,
  'vendor': 'Microsoft',
  'data': 'Invoice data for eZ3t54l0c5',
  'amount': 482.46,
  'category': 'Computers',
  'created_at': '2024-11-08T09:55:33.619896'},
 {'id': 6,
  'vendor': 'Lenovo',
  'data': 'Invoice data for 54g7YuZEFM',
  'amount': 8214.13,
  'category': 'Software',
  'created_at': '2024-04-09T09:55:

In [6]:
from sqlalchemy.orm import sessionmaker

SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()

session.query(Document).delete()
session.commit()
session.close()
